In [64]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from camel_tools.tokenizers.word import simple_word_tokenize
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

In [65]:
# Ensure necessary NLTK resources are downloaded
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [66]:
#Data cleaning functions
# Define text cleaning functions
def remove_links(text):
    url_pattern = r'https?://\S+|www\.\S+|/\S+-\S+/|\S+-\S+-\S+'
    return re.sub(url_pattern, '', text)

def short_doctor_names(text):
    doctor_name_pattern = r'الدكتور\s+(\w+)(?:\s+\w+){0,2}'
    return re.sub(doctor_name_pattern, lambda match: f'د.{match.group(1)}', text)

def remove_doctor_specialization(text):
    last_line_pattern = r'\n[^\n]+$'
    return re.sub(last_line_pattern, '', text)

def remove_duplicated_letters(text):
    duplicate_pattern = r'(.)\1{2,}'
    return re.sub(duplicate_pattern, r'\1\1', text)

def standardize_numbers(text):
    numerals_map = str.maketrans('٠١٢٣٤٥٦٧٨٩', '0123456789')
    return text.translate(numerals_map)

def remove_date_time(text):
    date_time_pattern = r'\b\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\b'
    return re.sub(date_time_pattern, '', text).strip()

def remove_punctuation(text):
    punctuation_pattern = r'[^\w\s\#]'
    return re.sub(punctuation_pattern, '', text)

def keep_arabic_numbers_punctuation(text):
    non_arabic_pattern = r'[^\u0600-\u06FF0-9\s\n.,;!?()،:؛؟"\':-]'
    return re.sub(non_arabic_pattern, '', text)

def remove_arabic_stop_words(text):
    arabic_stopwords = set(stopwords.words('arabic'))
    words = text.split()
    return ' '.join(word for word in words if word not in arabic_stopwords)

def standardize_arabic_alef(text):
    return text.replace('آ', 'ا').replace('إ', 'ا').replace('أ', 'ا')

def remove_all_numbers(text):
    number_pattern = r'\d+|[٠١٢٣٤٥٦٧٨٩]+'
    return re.sub(number_pattern, '', text)

In [67]:
train_df = pd.read_csv('/kaggle/input/cleandata/CleanData.csv')

In [68]:
# Apply cleaning functions to the 'question' and 'answer' columns
for column in ['question', 'answer']:
    train_df[column] = train_df[column].apply(remove_links).apply(short_doctor_names).apply(remove_doctor_specialization)
    train_df[column] = train_df[column].apply(remove_duplicated_letters).apply(standardize_numbers).apply(remove_date_time)
    train_df[column] = train_df[column].apply(remove_punctuation).apply(keep_arabic_numbers_punctuation)
    train_df[column] = train_df[column].apply(standardize_arabic_alef).apply(remove_all_numbers)
    train_df[column] = train_df[column].apply(remove_arabic_stop_words).apply(lambda x: ' '.join(x.split()))  # Removing extra spaces

In [69]:
# Tokenize questions for Word2Vec
train_df['tokenized_questions'] = train_df['question'].apply(simple_word_tokenize)

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=train_df['tokenized_questions'], vector_size=100, window=5, min_count=1, workers=4)


In [70]:
# Create average word vectors for questions
def average_word_vectors(words, model, num_features):
    feature_vector = np.zeros((num_features,), dtype="float64")
    nwords = 0
    for word in words:
        if word in model.wv:
            nwords += 1
            feature_vector = np.add(feature_vector, model.wv[word])
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
    return feature_vector

train_df['question_vectors'] = train_df['tokenized_questions'].apply(lambda x: average_word_vectors(x, word2vec_model, 100))


In [71]:
# Train TF-IDF model and normalize vectors
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(train_df['question'])
tfidf_matrix_normalized = normalize(tfidf_matrix)
word2vec_matrix = np.array(list(train_df['question_vectors']))
word2vec_matrix_normalized = normalize(word2vec_matrix)
combined_vectors = np.hstack((tfidf_matrix_normalized.toarray(), word2vec_matrix_normalized))


In [72]:
def find_closest_matches(query, num_matches=1, threshold=0.1):  # Example threshold, adjust based on testing
    query_tfidf = vectorizer.transform([query])
    query_tfidf_normalized = normalize(query_tfidf)
    query_tokens = simple_word_tokenize(query)
    query_word2vec = average_word_vectors(query_tokens, word2vec_model, 100)
    query_word2vec_normalized = normalize([query_word2vec])
    query_combined = np.hstack((query_tfidf_normalized.toarray(), query_word2vec_normalized))
    
    similarity_scores = cosine_similarity(query_combined, combined_vectors).flatten()
    max_score = np.max(similarity_scores)

    if max_score < threshold:
        return None
    else:
        closest_idxs = np.argsort(similarity_scores)[-num_matches:][::-1]
        return train_df.iloc[closest_idxs]


In [73]:
def answer_user_query():
    user_query = input("Please enter your medical question in Arabic: ")
    try:
        num_answers = int(input("How many answers would you like to see? "))
    except ValueError:
        print("Invalid number, defaulting to 1 answer.")
        num_answers = 1
    
    closest_matches = find_closest_matches(user_query, num_matches=num_answers)
    
    if closest_matches is None or closest_matches.empty:
        print("لا توجد اجابة قريبة يرجى كتابة سؤال اخر.")
    else:
        for idx, row in closest_matches.iterrows():
            print(f"\nQuestion: {row['question']}")
            print(f"Answer: {row['answer']}")
            print(f"Label: {row.get('label', 'N/A')}")
            print(" - - - - - - - - - - - - - - - - - - - - ")


In [74]:
answer_user_query()

Please enter your medical question in Arabic:  اسباب الشعور بالعطش
How many answers would you like to see?  ٣



Question: حرقة الشعور بالخاجة للحكة التبرز الشعور بان فتة الشرج مغلقة الم شديد يزداد الليل يحتمل عكس النهار يكون خفيف الا التبرز
Answer: ممكن تكون الحالة ناجمة شرخ شرجي التهابات جلدية راجع طبيب الجراحة للفحص داسامة اسامة العبدالله جراحة
Label: جراحة-عامة
 - - - - - - - - - - - - - - - - - - - - 

Question: ممرت الشعور بالدوخة مرتين اسبوع صعوبة بالتنفس يطل الشعور الغريب اكثر ثواني معدودة المرتين
Answer: ربما لديك التهاب حساسية الانف الجيوب الانفية واذا يكرر الامر فالارجح انه عارض هبوط ضغط بسبب طول الجلوس اللابتوب وانصح بالقيام والحركة دقيقة العمل المكتبي وكثرة شرب الماء قياس الضغط دنهى نهى دشاش طب الاسرة
Label: امراض-الجهاز-التنفسي
 - - - - - - - - - - - - - - - - - - - - 

Question: حاجة لشرح تحليل صورة الدم وتفسير اسباب الخانات المظللة تشير محتاجة اجراءات اخرى التحاليل غيرها الاجراءات الطبية العلم اني مدخنة
Answer: مبدئيا الفحص يشير وجود فقر دم يفضل المتابعه لمعرفه السبب دفراس فراس رباح باطنية
Label: امراض_الدم
 - - - - - - - - - - - - - - - - - - - - 


In [75]:
train_df.head(20)

,question,answer,label,tokenized_questions,question_vectors
0,يمكن لثوم يخفض الضغظ وماهي الجرعه المستخدمه ال...,يمكن وهناك كثير الدراسات تؤكد يمكن يكون بديلا ...,ارتفاع-ضغط-الدم,"[يمكن, لثوم, يخفض, الضغظ, وماهي, الجرعه, المست...","[-0.06601834894778828, 0.19500521422984698, -0..."
1,عملت فحص سكر الصائم صيام اكثر ساعة,النتيجة طبيعية داغيد داغيد الغدد الصماء,امراض_الجهاز_التنفسي,"[عملت, فحص, سكر, الصائم, صيام, اكثر, ساعة]","[-0.3580043998413852, 0.6460231755461011, -0.1..."
2,عملت ماموجرام وموجات صوتيه فوجدت تضخم العقد ال...,يفضل طبعا عمل رنين لانه ادق بكثير لمعرفه يجب ا...,الاورام-الخبيثة-والحميدة,"[عملت, ماموجرام, وموجات, صوتيه, فوجدت, تضخم, ا...","[0.13083218686282636, -0.25188932090997695, -0..."
3,عملت تحليل دم الاسبوع وخبرتني الدكتورة نشاط ال...,ظهور السكر بالبول متوقع الحمل الطبيعي يجب اجرا...,امراض الغدد الصماء,"[عملت, تحليل, دم, الاسبوع, وخبرتني, الدكتورة, ...","[-0.048066791478329554, 0.2012805391031389, 0...."
4,لديه اعراض تتمتل مشكلة التنفس عبارة اختناق لمد...,لابد تعرض حالته طبيب اطفال لاحتمال وجود حساسية...,الاورام_الخبيثة_والحميدة,"[لديه, اعراض, تتمتل, مشكلة, التنفس, عبارة, اخت...","[0.16086906676871643, 0.07694501805329515, -0...."
5,مريض عمره سنة شعر بضيق تنفس وتم عمل الايكو واش...,عليكماين الاشعات والتحاليل انها مرفقة دممدوح د...,امراض-الجهاز-التنفسي,"[مريض, عمره, سنة, شعر, بضيق, تنفس, وتم, عمل, ا...","[-0.9591443881392478, 0.20461587031604722, 0.5..."
6,عملت فحص سكر الصائم صيام اكثر ساعة,النتيجة طبيعية داغيد داغيد الغدد الصماء,امراض-الغدد-الصماء,"[عملت, فحص, سكر, الصائم, صيام, اكثر, ساعة]","[-0.3580043998413852, 0.6460231755461011, -0.1..."
7,استخدم جلوكوفاج والتراكمي,لابد اتباع نظام غذائى لمرضى السكر والمشى لمدة ...,مرض-السكري,"[استخدم, جلوكوفاج, والتراكمي]","[-0.020736129614912596, 0.07535729091614485, -..."
8,اجريت عملية توسيع قناة العمود الفقري الفقرتين ...,طبيعي المستحسن مراجعة الطبيب اجرى العملية البر...,جراحة-العظام,"[اجريت, عملية, توسيع, قناة, العمود, الفقري, ال...","[-0.009188741183606908, 0.06237295117850105, -..."
9,اني مصاب بداء السكر وااخذ جرع انسولين اليوم وع...,تحتاج تعديل جرعات الانسولين يتناسب وقرات السكر...,ارتفاع-ضغط-الدم,"[اني, مصاب, بداء, السكر, وااخذ, جرع, انسولين, ...","[-0.0580991117603844, 0.48860936766141094, -0...."
